In [1]:
!nvidia-smi

Fri Oct  7 22:34:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.31       Driver Version: 462.31       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1050   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   49C    P8    N/A /  N/A |    814MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

(device(type='cpu'),
 <torch.cuda.device at 0x12faa854880>)

## 查询可用 gpu 的数量

In [3]:
torch.cuda.device_count()

1

## 这两个函数允许我们在请求的 GPU 不存在的情况运行代码

In [5]:
def try_gpu(i=0):
    """如果存在，则返回 gpu(i)，否则返回 cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的 gpu，如果没有GPU，则返回 [cpu(),]"""
    devices = [
        torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus

(device(type='cuda', index=0),
 device(type='cpu'),
 <function __main__.try_all_gpus()>)

## 查询张量所在的设备

In [7]:
x = torch.tensor([1, 2, 3])
x.device # 默认存储在 cpu 上

device(type='cpu')

In [8]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [13]:
Y = torch.rand(2, 3, device=try_gpu())
Y

tensor([[0.0387, 0.0782, 0.7002],
        [0.5752, 0.3589, 0.9157]], device='cuda:0')

要计算 `X + Y`，我们需要决定在哪里执行这个操作

In [14]:
Z = X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


现在数据在同一个 GPU 上，（Z 和 Y 都在），我们可以将他们相加 

In [15]:
Y + Z

tensor([[1.0387, 1.0782, 1.7002],
        [1.5752, 1.3589, 1.9157]], device='cuda:0')

In [17]:
Z.cuda(0) is Z

True

 ## 神经网络与 GPU

In [18]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[0.5600],
        [0.5600]], device='cuda:0', grad_fn=<AddmmBackward>)

确认模型参数存储在同一个 GPU 上

In [19]:
net[0].weight.data.device

device(type='cuda', index=0)